# Qubit properties from capacitance simulations

In this notebook, we import the capacitance matrix computed in Elmer and write the resulting transmon Hamiltonian using [scQubits](https://scqubits.readthedocs.io/). The library may be installed with
```bash
pip install scqubits
```
ARM-based macOs might have problems with this, consult the scQubits documentation for workarounds.

Additionally, one needs the regular `numpy`, `scipy`, and `matplotlib`.


In [ ]:
import json
from pathlib import Path

import numpy as np
from matplotlib import pyplot as plt
import scqubits as scq

By default, the simulated results should be in the same folder as this notebook, or in some subdirectory. You may also point to the actual simulation folder in the `Path` object below.

In [ ]:
result_jsons = Path('./').rglob('*results.json')

# Let's look at one result
result_json = next(result_jsons)
with open(result_json) as file_pointer:
    result = json.load(file_pointer)
result

The matrix is given in a few formats in this case, let's use the `CMatrix`.

In [ ]:
result['CMatrix']

We simulated a case corresponding to the lumped-element model below:

![lumped-element model](floating_qubit_coupled.png)

For this case we should add a capacitive contribution of a full-length resonator to $C_{33}$. For $50\,\Omega$ and $f=6\,\text{GHz}$, this is roughly something like $450\,\text{fF}$. This will affect the $C_\Sigma$ little but will be relevant for estimating coupling $g$.

In [ ]:
result['CMatrix'][2][2] += 450e-15


def C_Sigma_two_islands(data):
    r""" Data argument is a dict with `CMatrix` key of a 3x3 capacitance matrix with a coupler as the last port.
    Derived with the Lagrangian. See the following references for similar derivations:
    
    [1] F. Marxer et al., “Long-distance transmon coupler with CZ gate fidelity above 99.8%”. arXiv:2208.09460, Dec. 19, 2022.
    [2] A. P. A. Cronheim, “A Circuit Lagrangian Formulation of Opto-mechanical Coupling between two Electrical Resonators mediated by a SQUID”.
        Delft University of Technology, Dec. 10, 2018. [Online]. Available: http://resolver.tudelft.nl/uuid:a4c72663-65c9-4857-8ffa-ebaf2cbc9782

    Returns:
        C_Sigma: Effective qubit total capacitance for :math:`C_\Sigma`.
        C_q: Qubit island–island capacitance with correction from coupler.
        C_r: Resonator capacitance with correction from qubit islands.
        C_qr: Effective coupling capacitance between the qubit and the resonator.
    """
    C_sim = data['CMatrix']
    C_theta = (C_sim[0][0] + C_sim[0][2] + C_sim[1][1] + C_sim[1][2])   
    C_q = C_sim[0][1] + ((C_sim[0][0] + C_sim[0][2]) * (C_sim[1][1] + C_sim[1][2])) / C_theta
    C_r = C_sim[0][2] + C_sim[1][2] + C_sim[2][2] - (C_sim[0][2] + C_sim[1][2]) ** 2 / C_theta
    C_qr = (C_sim[0][2] * C_sim[1][1] - C_sim[1][2] * C_sim[0][0]) / C_theta
    return C_q - C_qr ** 2 / C_r, C_q, C_r, C_qr

C_Sigma, C_q, C_r, C_qr = C_Sigma_two_islands(result)
print(f'{C_Sigma * 1e15} fF')

In [ ]:
from scipy.constants import e, h, pi, physical_constants

L_J = 7.015e-9  # Example from E. Hyyppä et al., ‘Unimon qubit’, Nat Commun, vol. 13, no. 1, Art. no. 1, Nov. 2022

# Units on Hz for energy (E_J / h)
E_J = (physical_constants['mag. flux quantum'][0] / (2*pi)) ** 2 / (h * L_J)
E_C = e ** 2 / (2 * h * C_Sigma)
print(f'ratio = {E_J / E_C}')

scq.set_units('Hz')  # Energy is now given in Hertz
transmon = scq.TunableTransmon(
    EJmax=E_J,
    EC=E_C,
    d=0.1,
    flux=0.95,
    ng=0.5,  # Default charge dispersion
    ncut=30,
)

transmon.plot_wavefunction(which=(0, 1, 2, 3), mode='abs_sqr');

In [ ]:
evals = transmon.eigenvals(3)
f_ge = evals[1] - evals[0]
f'Qubit frequency = {f_ge/1e9} GHz'

In [ ]:
transmon.plot_evals_vs_paramvals(param_name='flux', param_vals=np.linspace(0, 1, 100));
plt.title('Flux dependence')

fig, axes = transmon.plot_evals_vs_paramvals('ng', np.linspace(-2, 2, 100), evals_count=10, subtract_ground=False)
plt.title('Charge dispersion')

## Exercise

We can even try check the shift from coupling to a resonator. This is essentially the Jaynes—Cummings model but numerically.

1. Implement the missing parts below to compute anharmonicity $\alpha$ and the dispersive shift $\chi$ when connected to a $6\,\text{GHz}$ resonator.
    The formula for coupling is given by
    $$
    g \approx \frac{1}{2} \frac{C_\text{qr}}{\sqrt{C_{\Sigma} \left( C_\text{r} - \frac{C_\text{qr}^2}{C_\text{q}} \right) }}  \sqrt{\omega_\text{q}\omega_\text{r}}
    ,
    $$
    where $\omega_\alpha=2\pi f_\alpha$

2. Instead of simply adding $450\,\text{fF}$ to `C[2][2]`, add the correct value for a $Z_0=100\,\Omega$ $\lambda/4$ CPW resonator with a frequency of $6\,\text{GHz}$. You may use any online CPW calculators to get the properties. For a more programmable approach something like [`scikit-rf`](https://scikit-rf.readthedocs.io/en/latest/index.html) may be used. Use any reasonable values for the substrate properties.

In [ ]:
g = ...

resonator = scq.Oscillator(
    E_osc=...,
    truncated_dim=10
)

hilbert_space = scq.HilbertSpace([transmon, resonator])
# For details, see Eq. 3.3 in J. Koch et al., ‘Charge-insensitive qubit design derived from the Cooper pair box’,
#   Phys. Rev. A, vol. 76, no. 4, p. 042319, Oct. 2007
hilbert_space.add_interaction(
    g_strength=g,
    op1=transmon.n_operator,
    op2=resonator.creation_operator,
    add_hc=True
)

eigenvalues = hilbert_space.eigenvals(evals_count=20)

hilbert_space.generate_lookup()
g0 = eigenvalues[0]  # lowest state
e0 = eigenvalues[hilbert_space.dressed_index((1, 0))]  # qubit is excited, resonator is ground
f0 = eigenvalues[hilbert_space.dressed_index((2, 0))]
g1 = eigenvalues[hilbert_space.dressed_index((0, 1))]
e1 = eigenvalues[hilbert_space.dressed_index((1, 1))]

f_ge_coupled = ...
f_ef_coupled = ...
f_rr_coupled_g = ...
f_rr_coupled_e = e1 - e0

print(f'anharmonicity = {(f_ef_coupled - f_ge_coupled) / 1e6} MHz', f'chi = {0.5 * (f_rr_coupled_e - f_rr_coupled_g) / 1e6} MHz')
hilbert_space.hamiltonian()